<a href="https://colab.research.google.com/github/shubhamphal/One-NLP/blob/master/Text_Summarization_with_Universal_Sentence_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text

In [49]:
import numpy as np
import networkx as nx
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [50]:
universal_sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [51]:
def read_file(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    doc = filedata[0].split(". ")
    sentences = []
    for sentence in doc:
        sentences.append(sentence)
    return sentences

In [52]:
def sentence_similarity(sent1, sent2):
    emb_1 = universal_sentence_encoder([sent1]) 
    emb_2 = universal_sentence_encoder([sent2])
    return np.inner(emb_1, emb_2).flatten()[0]

In [53]:
def create_similarity_matrix(sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for sentence1 in range(len(sentences)):
        for sentence2 in range(len(sentences)):
            if sentence1 == sentence2:
                continue 
            similarity_matrix[sentence1][sentence2] = sentence_similarity(sentences[sentence1], sentences[sentence2])
    return similarity_matrix

In [56]:
def generate_summary(file_name, top_n=2):
    summarize_text = []
    sentences =  read_file(file_name)
    sentence_similarity_martix = create_similarity_matrix(sentences)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    
    for i in range(top_n):
        summarize_text.append("".join(ranked_sentence[i][1]))
    return ". ".join(summarize_text)

News Data url https://www.nytimes.com/2020/07/24/world/coronavirus-covid-19.html
Copy paste this data onto a text file



In [57]:
text_summary=generate_summary('nytimesnews.txt',3)

Indexes of top ranked_sentence order are  [(0.06740202825220068, 'The United States on Friday came just short of breaking its single-day record for new coronavirus cases, adding more than 73,400, the second-highest daily total, and signaling that infection rates show no signs of slowing'), (0.062188728531415646, 'South Korea, which has been held up as a success story, on Saturday reported 113 new infections, many of them imported'), (0.05919182208384038, 'County officials said there had been a rapid surge in both cases and hospitalizations in recent weeks'), (0.058873901880196824, 'Vietnam on Saturday confirmed a new infection, its first reported case of community transmission in 100 days'), (0.05580278205046402, 'As the number of cases has continued to climb, so has the number of hospitalizations, which had skirted its own record in recent days'), (0.052793191198487015, 'Since then, the situation has worsened across a number of states'), (0.05200019394622272, 'nears a single-day recor

In [58]:
print(text_summary)

The United States on Friday came just short of breaking its single-day record for new coronavirus cases, adding more than 73,400, the second-highest daily total, and signaling that infection rates show no signs of slowing. South Korea, which has been held up as a success story, on Saturday reported 113 new infections, many of them imported. County officials said there had been a rapid surge in both cases and hospitalizations in recent weeks
